In [1]:
# Libraries

from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd

In [2]:
# Selecting GPU 

device = "cuda"

In [3]:
# Initializing LLM

model_name = "Qwen/Qwen2.5-7B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

We've detected an older driver with an RTX 4000 series GPU. These drivers have issues with P2P. This can affect the multi-gpu inference when using accelerate device_map.Please make sure to update your driver to the latest version which resolves this.


In [4]:
# Sample dynamic DataFrame

data = {
    'id': [1, 2, 3],
    'name': ['Alice', 'Bob', 'Charlie'],
    'age': [25, 30, 35],
    'department': ['sales', 'marketing', 'sales']
}
df = pd.DataFrame(data)

In [5]:
def get_llm_response(model, tokenizer, prompt):

    system_prompt = """
    You are a SQL generation assistant. You will receive:
    1. A question written in natural language.
    2. A database schema with table names and columns.

    Your task is to generate an accurate and valid SQL query based on the question, strictly adhering to the following guidelines:

    Instructions for SQL Query Generation:
    - Output Format: Output only the SQL query without any extra text or explanation.
    - Schema Matching: Use only the table names and columns specified in the schema provided. Do not assume or invent any columns.
    - Table and Column Names: Refer to all table and column names exactly as they appear in the schema, including case-sensitivity if specified.
    - Joins and Relationships: If the query requires data from multiple tables, assume natural joins where primary and foreign key relationships are implied.
    - Conditions: Use WHERE clauses for conditions specified in the question. Include logical operators (AND, OR) if multiple conditions are mentioned.
    - Aggregations: For questions involving calculations (e.g., total, average, count), use appropriate aggregate functions (e.g., SUM, AVG, COUNT) only if specifically mentioned or implied.
    - Ordering and Limiting: If the question implies sorting (e.g., "highest," "lowest," "most recent"), use ORDER BY. Use LIMIT only if a specific number of results is requested or implied.
    - Output Columns: Include only the columns relevant to answering the question. If no specific columns are requested, return all columns.

    Error Handling:
    - If any column or table required by the question is missing in the schema, do not proceed. Instead, return an error message: Error: Column or table not found in schema.

    Examples:
    1. Schema:
       - Table: employees
         - id (INTEGER)
         - name (TEXT)
         - age (INTEGER)
         - department (TEXT)

       Question: "List the names of employees who are older than 30."
       Output: SELECT name FROM employees WHERE age > 30;

    2. Schema:
       - Table: sales
         - id (INTEGER)
         - product_name (TEXT)
         - amount (FLOAT)
         - sale_date (DATE)

       Question: "Find the total sales amount."
       Output: SELECT SUM(amount) FROM sales;

    Provide only the SQL query.
    """
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)
    
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response= tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response.strip()

In [6]:
# Function to dynamically get schema from DataFrame

def get_dataframe_schema(df):
    schema = "Table: dynamic_table\n"
    for column, dtype in df.dtypes.items():
        schema += f"- {column} ({dtype.name.upper()})\n"
    return schema

In [7]:
# Convert NL to SQL using an LLM

def nl_to_sql(model, tokenizer, nl_question, schema):
    prompt = f"Convert the following question into an SQL query.\n\nSchema:\n{schema}\n\nQuestion: \"{nl_question}\"\n\nSQL:"
    sql_query = get_llm_response(model, tokenizer, prompt)
    return sql_query

In [8]:
# Natural language question

nl_question = "Get the names of employees older than 30 who work in the sales department."

In [9]:
# Generate schema dynamically

schema = get_dataframe_schema(df)
print("Generated Schema:\n", schema)

Generated Schema:
 Table: dynamic_table
- id (INT64)
- name (OBJECT)
- age (INT64)
- department (OBJECT)



In [11]:
# Generate SQL query

sql_query = nl_to_sql(model, tokenizer, nl_question, schema)
print("Generated SQL Query:", sql_query)

Generated SQL Query: SELECT name FROM dynamic_table WHERE age > 30 AND department = 'sales';
